> This is the notebook for the milestone 2 of the project of the pizzafels team! We propose to study the car accident in Swiss

# DESCRIBE THE DATA

### .Imports

In [2]:
import os
import pandas as pd
import json
import matplotlib.pyplot as plt
import numpy as np
import re
import math
import numpy as np

%pylab inline
pylab.rcParams['figure.figsize'] = (20,12)

Populating the interactive namespace from numpy and matplotlib


### .Constants

In [3]:
DATA_FOLDER = 'data_clean'
IS_DATE = re.compile("^[1-2]{1}[0-9]{3}$") #will be useful to detect values that correspond to dates

### .Usefull functions

#### Function to quickly describe a dataframe

In [74]:
def describeDf(df,name="DESCRIPTION",level=2):
    date=[]
    if(level==2):
        print("______________________________ " + name+" ______________________________\n")
        print("\n")
        for col in df.columns:
            if((not IS_DATE.match(col)) & (not re.compile("[0-9]+ an").match(col))):
                print("         ATTRIBUTE: "+col)
                print("   "+str(df[col].unique()))
                print("\n")
            else:
                date.append(col)
        print("         VALUES: "+str(date))
        print("\n\n\n")
        return
    if(level==1):
        print("______________________________ " + name+" ______________________________\n")
        val = []
        for col in df.columns:
            if((not IS_DATE.match(col)) & (not re.compile("[0-9]+ an").match(col))):
                val.append(col)
        print(val)
        print("\n\n\n")
        return
    print("   >  " + name+"\n")

### .Load Datasets

> We took many datasets from the website OpenSwissData. All these datasets are excels spreedsheet in differents forms so we did a pre-treatment to transform all the datasets into a clear csv. All these csv files are store in the folder `data_clean`.

> The pre-treatment was huge since sometime datasets where split into multiple spreedsheet (on for each year) for example. Moreover, the excel files were organized to be visually good looking, with many merged cells and various hierachical levels in the data, thus requiring more work to clean it ready for processing with pandas. 

In [64]:
# print list of used dataframes
dataframes = {}
print("  DATAFRAMES AVAILABLES:\n")
for file in os.listdir(DATA_FOLDER):
    filename = os.fsdecode(file)
    tablename = "df_"+os.path.splitext(os.path.splitext(filename)[0])[0]
    globals()[tablename] = pd.read_csv(DATA_FOLDER + '/' + filename)
    try:
        globals()[tablename] = globals()[tablename].drop(columns=['Unnamed: 0'])
    except:
        pass
    describeDf(globals()[tablename], name=tablename,level=0)
    dataframes[tablename] = globals()[tablename]
    
    

  DATAFRAMES AVAILABLES:

   >  df_Accident_cantons

   >  df_Accident_circonstances

   >  df_Accident_objets

   >  df_Accident_type_route

   >  df_Accident_victimes

   >  df_Besoin_sante

   >  df_Depense_menages_canton

   >  df_Frais_routes_cantonales

   >  df_Frais_routes_communales

   >  df_Frais_routes_nationales

   >  df_Longueur_routes

   >  df_Population_2010

   >  df_Population_age_1992

   >  df_Population_canton_1992

   >  df_Proportion_permis

   >  df_Qualite_vie_agglo

   >  df_Recettes_routes

   >  df_Voitures



### .Descriptions Datasets

In [90]:
describedDatasets = []

#### Accidents

The following datasets give information about accidents

In [91]:
#print a list of the data contained in each used datasets concerning accidents in Switzerland (categories in french)
regex_accident = re.compile('.*Accident.*')
for index in dataframes:
    if regex_accident.match(index):
        describeDf(dataframes[index],index,level=1)
        describedDatasets.append(index)

______________________________ df_Accident_cantons ______________________________

['canton_index', 'canton', 'gravite_index', 'gravite', 'ruralite_index', 'ruralite']




______________________________ df_Accident_circonstances ______________________________

['mois_index', 'mois', 'day_index', 'day', 'hours_index', 'hours', 'gravite_index', 'gravite', 'ruralite_index', 'ruralite', 'contexte_index', 'contexte', 'circonstance_index', 'circonstance']




______________________________ df_Accident_objets ______________________________

['voiture_index', 'voiture', 'age_index', 'age', 'sexe_index', 'sexe', 'annee_permis_index', 'annee_permis', 'faute_index', 'faute', 'gravite_index', 'gravite']




______________________________ df_Accident_type_route ______________________________

['gravite_index', 'gravite', 'ruralite_index', 'ruralite', 'voie_index', 'voie', 'vitesse_index', 'vitesse', 'temps_index', 'temps', 'signalisation_index', 'signalisation']




______________________________ d

> As we can see, we have agregate data about accident in Switzerland. Since we don't have the original dataset of all car accidents in Swiss, we can't make some link between these dataset.

> But we asked to the Swiss Federal Office of Statistics to get a brut dataset of car accidents per ID. It could bring us more information to perform our data analysis

#### Cantons

The following datasets gives information for each canton

In [98]:
#print list of used datasets about Swiss cantons for further correlations with accidents
regex_canton = re.compile('.*[C|c]anton.*')
regex_population = re.compile('.*Population.*')
for index in dataframes:
    if not regex_population.match(index):
        for col in dataframes[index].columns:
            if regex_canton.match(col):
                describeDf(dataframes[index],index,level=1)
                describedDatasets.append(index)
                break

______________________________ df_Accident_cantons ______________________________

['canton_index', 'canton', 'gravite_index', 'gravite', 'ruralite_index', 'ruralite']




______________________________ df_Besoin_sante ______________________________

['annee', 'canton', 'Total', 'Soins hopitaux', 'Soins domicile', 'Administration Prevention', 'besoin_par_habitant']




______________________________ df_Depense_menages_canton ______________________________

['annee', 'Canton', 'Mode', 'Eaux-de-vie', 'Vins', 'Bières', 'Boissons alcoolisées dans les restaurants, cafés et bars', 'Boissons alcoolisées en petite restauration', 'Boissons alcoolisées dans les cantines', "Boissons alcoolisées chez des privés lors d'invitations", 'Automobiles', 'Motos, scooters et vélomoteurs', 'Vélos', 'Carburants et lubrifiants', 'Train, billets et abonnements', 'Tram, billets et abonnements', 'Bus, billets et abonnements', 'Bateau, billets et abonnements', 'Transport multimodal', 'Services de téléphonie mobil

#### Population

The following datasets give information about Swiss population and also the proportion of population who had a driver licence

In [93]:
# print list of datasets about Swiss demography to further normalize our data about drivers profiles
regex_population = re.compile('.*(Population|permis).*')
for index in dataframes:
    if regex_population.match(index):
        describeDf(dataframes[index],index,level=1)
        describedDatasets.append(index)

______________________________ df_Population_2010 ______________________________

['annee', 'canton_index', 'canton', 'permanente', 'sexe', 'etat_civil', 'Age - Total']




______________________________ df_Population_age_1992 ______________________________

['age', 'sexe']




______________________________ df_Population_canton_1992 ______________________________

['canton', 'sexe']




______________________________ df_Proportion_permis ______________________________

['critere']






In [94]:
print('df_Population_2010')
display(df_Population_2010.head(2))
print('\n\n\n\ndf_Population_canton_1992')
display(df_Population_canton_1992.head(2))
print('\n\n\ndf_Population_age_1992')
display(df_Population_age_1992 .head(2))

df_Population_2010


,annee,canton_index,canton,permanente,sexe,etat_civil,Age - Total,0 ans,1 an,2 ans,...,91 ans,92 ans,93 ans,94 ans,95 ans,96 ans,97 ans,98 ans,99 ans,100 ans ou plus
0,2010.0,8100,Suisse,Population résidante permanente,Sexe - Total,Etat civil - Total,7870134.0,78031.0,79351.0,78793.0,...,10569.0,8089.0,6540.0,5002.0,3765.0,3071.0,2114.0,1413.0,905.0,1332.0
1,2010.0,8100,Suisse,Population résidante permanente,Sexe - Total,Célibataire,3385597.0,78031.0,79351.0,78793.0,...,943.0,680.0,560.0,498.0,389.0,321.0,203.0,146.0,90.0,175.0






df_Population_canton_1992


,canton,sexe,1992,1993,1994,1995,1996,1997,1998,1999,...,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010
0,Suisse,Sexe - Total,6943095,6988858,7036852,7080948,7105446,7113373,7131888,7166738,...,7285214,7342981,7405051,7454112,7501255,7557609,7618599,7711056,7801278,7877571
1,Suisse,Homme,3409670,3426199,3447614,3467945,3477147,3478179,3486789,3505087,...,3569257,3599446,3632015,3657517,3682520,3714697,3748587,3798705,3846081,3885339





df_Population_age_1992


,age,sexe,1992,1993,1994,1995,1996,1997,1998,1999,...,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010
0,Age - Total,Sexe - Total,6943095,6988858,7036852,7080948,7105446,7113373,7131888,7166738,...,7285214,7342981,7405051,7454112,7501255,7557609,7618599,7711056,7801278,7877571
1,Age - Total,Homme,3409670,3426199,3447614,3467945,3477147,3478179,3486789,3505087,...,3569257,3599446,3632015,3657517,3682520,3714697,3748587,3798705,3846081,3885339


> If we look at these two datasets, we can see that one go from 1992 to 2009 and the other one from 2010 to 2017. The problem is they are in different format, one get the date in column and the other one get the age in column.

> There are two datasets between 1992 and 2009 since one give us information about the sexe and the other one about the canton.

# PLAN FOR ANALYSIS

> Our aim is to give advice to the Swiss government and cantons to make the roads safer!

## 1. Description of car accidents context in Switzerland

This section is aimed at describing responsible actors and victims in car accidents, as well as various related informations to find insights that might be helpful to decision makers. 

### 1.1. Who?

#### 1.1.1. Sexe and age

In [102]:
# Using df_Accident_objets and also populations datasets to normalize
display(df_Accident_objets.head(2))
describeDf(df_Accident_objets)

,voiture_index,voiture,age_index,age,sexe_index,sexe,annee_permis_index,annee_permis,faute_index,faute,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,210.0,Voiture de tourisme,0.0,Jusqu'à 4 ans,1.0,Homme,0.0,0 an,0.0,Objet sans faute ni influence,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,210.0,Voiture de tourisme,0.0,Jusqu'à 4 ans,1.0,Homme,0.0,0 an,0.0,Objet sans faute ni influence,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


______________________________ DESCRIPTION ______________________________



         ATTRIBUTE: voiture_index
   [210. 211. 214. 220. 221. 222. 223. 224. 227. 228. 229.]


         ATTRIBUTE: voiture
   ['Voiture de tourisme' 'Véhicule de transport de personnes'
 'Véhicule de transport de choses' 'Cycle' 'Cyclomoteur' 'Motocycle léger'
 "Motocycle jusqu'à 125 cm3" 'Motocycle de plus de 125 cm3' 'Piéton'
 'Autre véhicule non motorisé' 'Autre et inconnu']


         ATTRIBUTE: age_index
   [  0.   5.   6.  10.  15.  18.  20.  21.  30.  40.  50.  60.  70. 999.]


         ATTRIBUTE: age
   ["Jusqu'à 4 ans" '5 ans' '6-9 ans' '10-14 ans' '15-17 ans' '18-19 ans'
 '20 ans' '21-29 ans' '30-39 ans' '40-49 ans' '50-59 ans' '60-69 ans'
 '70 ans et plus' 'Inconnu']


         ATTRIBUTE: sexe_index
   [1. 2. 9.]


         ATTRIBUTE: sexe
   ['Homme' 'Femme' 'Inconnu']


         ATTRIBUTE: annee_permis_index
   [  0.   1.   2.   3.   4.   5.   6.  11.  16. 999.]


         ATTRIBUTE: annee_permis

The previous dataset will be key to describe the profile of drivers according to sex and age, and provide their chances of being responsible of car accidents. Since we have observed that car accidents caused by women have first increased and then decreased, an animated bar plot could be interesting to visualize the difference of exposure of women and men to car accidents responibility throughout years. 

#### 1.1.2. Driver Licence

In [ ]:
# Using df_Accident_objets and also populations datasets to normalize as above

### 1.2. Where?

#### 1.2.1. Road type

In [104]:
# Using df_Accident_type_route 
display(df_Accident_type_route.head(2))
describeDf(df_Accident_type_route)

,gravite_index,gravite,ruralite_index,ruralite,voie_index,voie,vitesse_index,vitesse,temps_index,temps,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,315.0,Accidents avec blessés légers,10.0,En localité,50.0,Autoroute,30.0,10-30 Km/h,110.0,Sec,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,315.0,Accidents avec blessés légers,10.0,En localité,50.0,Autoroute,30.0,10-30 Km/h,110.0,Sec,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


______________________________ DESCRIPTION ______________________________



         ATTRIBUTE: gravite_index
   [315. 316. 317.]


         ATTRIBUTE: gravite
   ['Accidents avec blessés légers' 'Accidents avec blessés graves'
 'Accidents mortels']


         ATTRIBUTE: ruralite_index
   [10. 11. 12.]


         ATTRIBUTE: ruralite
   ['En localité' 'Hors localité' 'Sur autoroute']


         ATTRIBUTE: voie_index
   [50. 51. 52. 53. 54.]


         ATTRIBUTE: voie
   ['Autoroute' 'Semi-autoroute' 'Route principale' 'Route secondaire'
 'Autre route']


         ATTRIBUTE: vitesse_index
   [ 30.  50.  60.  80. 120.]


         ATTRIBUTE: vitesse
   ['10-30 Km/h' '40-50 Km/h' '60 Km/h' '70-80 Km/h' '90-120 Km/h']


         ATTRIBUTE: temps_index
   [110. 111. 112. 113. 114. 115. 116.]


         ATTRIBUTE: temps
   ['Sec' 'Humide' 'Mouillé' 'Enneigé' 'Verglacé' 'Neige fondante'
 'Autre état de la route']


         ATTRIBUTE: signalisation_index
   [150. 151. 152. 153. 154. 155. 156. 

This dataset provides many informations about the riskiness of different driving environments, and it will be studied to provide potential insights to decision makers at national scale. 

#### 1.2.2. Cantons

This part describes the main dataset that we plan to use in the scope of a 'canton-scale comparative study'. Since we have an **aggregated** dataset of car accidents in cantons, the possibility of correlations within the dataset are limited. 

In [103]:
# Using df_Accident_cantons
display(df_Accident_cantons.head(2))
describeDf(df_Accident_cantons)

,canton_index,canton,gravite_index,gravite,ruralite_index,ruralite,1992,1993,1994,1995,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,1.0,Zürich,315.0,Accidents avec blessés légers,10.0,En localité,1323.0,1514.0,1752.0,1629.0,...,2035.0,2020.0,1915.0,1874.0,1789.0,1696.0,1877.0,2039.0,2247.0,2161.0
1,1.0,Zürich,315.0,Accidents avec blessés légers,11.0,Hors localité,388.0,341.0,366.0,360.0,...,354.0,395.0,358.0,315.0,321.0,314.0,340.0,353.0,352.0,419.0


______________________________ DESCRIPTION ______________________________



         ATTRIBUTE: canton_index
   [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 19. 20. 21. 22. 23. 24. 25. 26.]


         ATTRIBUTE: canton
   ['Zürich' 'Bern / Berne' 'Luzern' 'Uri' 'Schwyz' 'Obwalden' 'Nidwalden'
 'Glarus' 'Zug' 'Fribourg / Freiburg' 'Solothurn' 'Basel-Stadt'
 'Basel-Landschaft' 'Schaffhausen' 'Appenzell Ausserrhoden'
 'Appenzell Innerrhoden' 'St. Gallen' 'Graubünden / Grigioni / Grischun'
 'Aargau' 'Thurgau' 'Ticino' 'Vaud' 'Valais / Wallis' 'Neuchâtel' 'Genève'
 'Jura']


         ATTRIBUTE: gravite_index
   [315. 316. 317.]


         ATTRIBUTE: gravite
   ['Accidents avec blessés légers' 'Accidents avec blessés graves'
 'Accidents mortels']


         ATTRIBUTE: ruralite_index
   [10. 11. 12.]


         ATTRIBUTE: ruralite
   ['En localité' 'Hors localité' 'Sur autoroute']


         VALUES: ['1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999',

We see that this dataset is providing informations about the rurality (motorway, city road, country road), which can be used to study correlations with informations about new road length in each canton for example. 
However, this dataset is very limited in terms of features. Therefore, correlations should be studied between various features obtained from other canton-scale datasets over last years, and corresponding car accidents from this dataset.

### 1.3. When?

In [105]:
# Using df_Accident_circonstances 
display(df_Accident_circonstances.head(2))
describeDf(df_Accident_circonstances)

,mois_index,mois,day_index,day,hours_index,hours,gravite_index,gravite,ruralite_index,ruralite,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,1.0,Janvier,1.0,Lundi,0.0,00-05 heures,315.0,Accidents avec blessés légers,10.0,En localité,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,Janvier,1.0,Lundi,0.0,00-05 heures,315.0,Accidents avec blessés légers,10.0,En localité,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


______________________________ DESCRIPTION ______________________________



         ATTRIBUTE: mois_index
   [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]


         ATTRIBUTE: mois
   ['Janvier' 'Février' 'Mars' 'Avril' 'Mai' 'Juin' 'Juillet' 'Août'
 'Septembre' 'Octobre' 'Novembre' 'Décembre']


         ATTRIBUTE: day_index
   [1. 2. 3. 4. 5. 6. 7.]


         ATTRIBUTE: day
   ['Lundi' 'Mardi' 'Mercredi' 'Jeudi' 'Vendredi' 'Samedi' 'Dimanche']


         ATTRIBUTE: hours_index
   [ 0.  6. 12. 18. 99.]


         ATTRIBUTE: hours
   ['00-05 heures' '06-11 heures' '12-17 heures' '18-24 heures'
 'Heure inconnue']


         ATTRIBUTE: gravite_index
   [315. 316. 317.]


         ATTRIBUTE: gravite
   ['Accidents avec blessés légers' 'Accidents avec blessés graves'
 'Accidents mortels']


         ATTRIBUTE: ruralite_index
   [10. 11. 12.]


         ATTRIBUTE: ruralite
   ['En localité' 'Hors localité' 'Sur autoroute']


         ATTRIBUTE: contexte_index
   ['A' 'C' 'D' 'E' 'G'

Previous dataset might provide some relevant outcomes regarding decision makers. 

### 1.4. What? (victims)

In [106]:
# Using df_Accident_victimes
display(df_Accident_victimes.head(2))
describeDf(df_Accident_victimes)

,voiture_index,voiture,sexe_index,sexe,role_index,role,age_index,age,consequence_index,consequence,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,210.0,Voiture de tourisme,1.0,Homme,300.0,Conducteur,0.0,jusqu'à 4 ans,315.0,Blessé léger,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,210.0,Voiture de tourisme,1.0,Homme,300.0,Conducteur,0.0,jusqu'à 4 ans,316.0,Blessé grave,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


______________________________ DESCRIPTION ______________________________



         ATTRIBUTE: voiture_index
   [210. 211. 214. 220. 221. 222. 223. 224. 227. 228. 229.]


         ATTRIBUTE: voiture
   ['Voiture de tourisme' 'Véhicule de transport de personnes'
 'Véhicule de transport de choses' 'Cycle' 'Cyclomoteur' 'Motocycle léger'
 "Motocycle jusqu'à 125 cm3" 'Motocycle de plus de 125 cm3' 'Piéton'
 'Autre véhicule non motorisé' 'Autre et inconnu']


         ATTRIBUTE: sexe_index
   [1. 2. 9.]


         ATTRIBUTE: sexe
   ['Homme' 'Femme' 'Inconnu']


         ATTRIBUTE: role_index
   [300. 301. 303.]


         ATTRIBUTE: role
   ['Conducteur' 'Piéton' 'Passager']


         ATTRIBUTE: age_index
   [  0.   5.   6.  10.  15.  18.  20.  21.  30.  40.  50.  60.  70. 999.]


         ATTRIBUTE: age
   ["jusqu'à 4 ans" '5 ans' '6-9 ans' '10-14 ans' '15-17 ans' '18-19 ans'
 '20 ans' '21-29 ans' '30-39 ans' '40-49 ans' '50-59 ans' '60-69 ans'
 '70 ans et +' 'Inconnu']


         ATTR

One more time, since decision makers want to minimize the number of victims, some outcoming correlations with vehicle type for example might be useful in our study. 

## 2. Correlations focused on cantons-granulated data

> As explained above, cantons are interesting data to study car accidents in different contexts, and therefore finding potential correlations between various factors and the number of accidents per canton. The following datasets have been selected because they are linked to road safety and good operation, such as investment in road sector, expenses of households in alcool and public transport as well as type of cars prefered in each canton. 

### 2.1. Expenses for Roads

In [115]:
# Using df_Frais_routes_cantonales, df_Frais_routes_communales, df_Frais_routes_nationales
display(df_Frais_routes_cantonales.head(2))

,annee,canton,Total,Constructions,Amélio-,Acquisition,Entretien Construction,Entretien Exploitation,Administration,Surveillance Policiere
0,1994,Appenzell Rh.-Ext.,31374.0,0,16298,-46,2796,7706,1705.0,2915
1,1994,Appenzell Rh.-Int.,4206.0,488,931,0,265,1107,887.0,527


### 2.2. Road length

In [120]:
# Using df_Longueur_routes
display(df_Longueur_routes.head(2))

,annee,canton,Total,Routes nationales,Routes cantonales,Routes communales
0,1992,Appenzell Rh.-Ext.,434.0,0.0,231.0,203.0
1,1992,Appenzell Rh.-Int.,139.0,0.0,48.0,91.0


### 2.3. Police

In [122]:
# Using df_Frais_routes_cantonales, df_Frais_routes_communales, df_Frais_routes_nationales with "Surveillance Policiere"
display(df_Frais_routes_cantonales.head(2))

,annee,canton,Total,Constructions,Amélio-,Acquisition,Entretien Construction,Entretien Exploitation,Administration,Surveillance Policiere
0,1994,Appenzell Rh.-Ext.,31374.0,0,16298,-46,2796,7706,1705.0,2915
1,1994,Appenzell Rh.-Int.,4206.0,488,931,0,265,1107,887.0,527


### 2.4. Type of cars

In [123]:
# Using df_Voitures
display(df_Voitures.head(2))
describeDf(df_Voitures)

,canton_index,canton,carosserie_index,carosserie,carburant_index,carburant,cylindre_index,cylindre,traction_index,traction,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,1.0,Zürich,11.0,Limousine,100.0,Essence,101399.0,jusqu'à 1399 cm3,1.0,Traction avant,...,9465.0,9462.0,9364.0,9276.0,9375.0,9315.0,9448.0,9474.0,9881.0,10412.0
1,1.0,Zürich,11.0,Limousine,100.0,Essence,101399.0,jusqu'à 1399 cm3,1.0,Traction avant,...,86546.0,87274.0,91180.0,96187.0,100565.0,105378.0,109020.0,113222.0,115168.0,116336.0


______________________________ DESCRIPTION ______________________________



         ATTRIBUTE: canton_index
   [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 19. 20. 21. 22. 23. 24. 25. 26. 27.]


         ATTRIBUTE: canton
   ['Zürich' 'Bern / Berne' 'Luzern' 'Uri' 'Schwyz' 'Obwalden' 'Nidwalden'
 'Glarus' 'Zug' 'Fribourg / Freiburg' 'Solothurn' 'Basel-Stadt'
 'Basel-Landschaft' 'Schaffhausen' 'Appenzell Ausserrhoden'
 'Appenzell Innerrhoden' 'St. Gallen' 'Graubünden / Grigioni / Grischun'
 'Aargau' 'Thurgau' 'Ticino' 'Vaud' 'Valais / Wallis' 'Neuchâtel' 'Genève'
 'Jura' 'Confédération']


         ATTRIBUTE: carosserie_index
   [11. 12. 13.]


         ATTRIBUTE: carosserie
   ['Limousine' 'Break' 'Cabriolet']


         ATTRIBUTE: carburant_index
   [ 100.  200.  300. 9900.]


         ATTRIBUTE: carburant
   ['Essence' 'Diesel' 'Electrique' 'Autres']


         ATTRIBUTE: cylindre_index
   [ 1.01399e+05  1.01799e+05  1.01999e+05  1.02499e+05  1.09999e+0

### 2.5. Expenses of citizens

In [125]:
# Using df_Depense_menages_canton
df_Depense_menages_canton.head(2)

,annee,Canton,Mode,Eaux-de-vie,Vins,Bières,"Boissons alcoolisées dans les restaurants, cafés et bars",Boissons alcoolisées en petite restauration,Boissons alcoolisées dans les cantines,Boissons alcoolisées chez des privés lors d'invitations,...,"Motos, scooters et vélomoteurs",Vélos,Carburants et lubrifiants,"Train, billets et abonnements","Tram, billets et abonnements","Bus, billets et abonnements","Bateau, billets et abonnements",Transport multimodal,Services de téléphonie mobile,Assurance véhicule
0,2006–2008,Zurich,Tot,8.238019489450405,56.251273,8.361510,75.590536,6.067617,0.37909465523331387,( ),...,( ),11.510053967140562,141.489994,34.573547,4.047249522742667,3.842554,3.172501021756669,72.360688,64.300854,101.943344
1,2006–2008,Zurich,Pct,0.0008621494018315626,0.005887,0.000875,0.007911,0.000635,3.967412685360722e-05,( ),...,( ),0.0012045839604442686,0.014808,0.003618,0.0004235646394821113,0.000402,0.00033201789115942093,0.007573,0.006729,0.010669


### 2.6. Impact of the health budget

In [126]:
# Using df_Besoin_sante
df_Besoin_sante.head(2)

,annee,canton,Total,Soins hopitaux,Soins domicile,Administration Prevention,besoin_par_habitant
0,1996,Appenzell Rh.-Ext.,38.845225,37.085203,0.176061,1.583962,751.683415
1,1996,Appenzell Rh.-Int.,4.794571,4.155756,0.363790,0.275025,456.258563


## 3. Predictions

> For each feature, we plan to measure whether they are correlated to car accidents frequency or not. One approach might be the Pearson coorelation coefficient, however, we could also study if another distribution than normal law is revealed. For the well correlated features (low p Value of uncorrelation hypothesis), we could use machine learning and apply a regression to predict the potential number of car accidents following different possible decisions made in investment and road infrastructure by a canton for example.

## 4. Conclusion

> Our recommendations to decision makers based on the results of the study!